# 🎧 QualiVault: Prepare Audio
**Goal:** Convert raw audio files to 16-bit 16kHz FLAC, analyze channel separation, and count speakers.

1. Reads `processing_recipe.yaml`.
2. Converts/Merges audio files using `ffmpeg`.
3. Saves them to the configured `flac_output_folder` (outside the git repo).
4. **Analyzes Stereo Separation:** Checks if Left/Right channels are distinct.
5. **Counts Speakers:** Uses Pyannote AI to estimate speakers per channel.
6. Updates `processing_recipe.yaml` with the analysis results.

In [ ]:
%load_ext autoreload
%autoreload 2
import yaml
import torch
import warnings
from pathlib import Path
import IPython.display as ipd
from pyannote.audio import Pipeline
from qualivault.audio import prepare_audio, analyze_channel_separation

# 1. Load Configuration
project_root = Path("..").resolve()
config_path = project_root / "config.yml"

with open(config_path) as f:
    config = yaml.safe_load(f)

# Suppress warnings if configured
if config.get("suppress_warnings", True):
    warnings.filterwarnings("ignore")

# 2. Setup Output Folder
# Resolve path relative to project root
output_dir = (project_root / config['paths']['flac_output_folder']).resolve()
output_dir.mkdir(parents=True, exist_ok=True)
print(f"📂 FLAC Output Directory: {output_dir}")

In [ ]:
# 3. Load Recipe
recipe_path = project_root / "processing_recipe.yaml"
if not recipe_path.exists():
    print("❌ Recipe not found! Please run '00_Setup_and_Scan.ipynb' first.")
    recipe = []
else:
    with open(recipe_path) as f:
        recipe = yaml.safe_load(f)
    print(f"✅ Loaded recipe with {len(recipe)} interviews.")

In [ ]:
# 4. Initialize AI Pipeline (Pyannote)
hf_token = config.get('hf_token')
pipeline = None

if hf_token:
    try:
        print("⏳ Initializing Pyannote Pipeline...")
        pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", 
                                            use_auth_token=hf_token)
        
        # Move to GPU/MPS if available
        device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
        pipeline.to(device)
        print(f"✅ Pyannote initialized on {device}")
    except Exception as e:
        print(f"⚠️ Could not initialize Pyannote: {e}")
else:
    print("⚠️ No HF_TOKEN found in config. Skipping speaker counting.")

In [ ]:
# 5. Process Audio & Analyze
processed_paths = {}
for item in recipe:
    i_id = item['id']
    input_files = item['files']
    output_name = item['output_name']
    output_path = output_dir / output_name
    
    print(f"\n🎧 Processing Interview: {i_id}")
    
    if output_path.exists():
        print(f"  ✅ File already exists: {output_path.name}")
        processed_paths[i_id] = output_path
    else:
        try:
            prepare_audio(input_files, output_path)
            print(f"  ✅ Converted to FLAC: {output_path.name}")
            processed_paths[i_id] = output_path
        except Exception as e:
            print(f"  ❌ Error: {e}")
            continue
    
    # Run Analysis
    if output_path.exists():
        analysis_stats = analyze_channel_separation(output_path, pipeline)
        # Update recipe item with stats
        item.update(analysis_stats)

In [ ]:
# 6. Save Updated Recipe
with open(recipe_path, 'w') as f:
    yaml.dump(recipe, f, sort_keys=False)
print(f"\n💾 Updated recipe saved to {recipe_path}")
print("   (You can now manually edit speaker counts in the YAML if needed)")

### 7. Quality Check
Loading all audio files at once can crash the kernel. Use the cell below to play specific files.

In [ ]:
# Select an interview ID to play (change the index or ID string)
if processed_paths:
    # Example: Play the first one
    target_id = list(processed_paths.keys())[0]
    target_path = processed_paths[target_id]
    
    print(f"▶️ Playing: {target_id} ({target_path.name})")
    display(ipd.Audio(filename=target_path))
else:
    print("No files processed.")